In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Selenium 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 결과를 저장할 딕셔너리
results = {}

try:
    url = "https://coinmarketcap.com/cryptocurrency-category/"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기
    
    # <tr> 요소 찾기
    tr_elements = driver.find_elements(By.CSS_SELECTOR, 'tr[style*="cursor:pointer"]')
    # 각 <tr> 요소 안의 <a> 태그에서 href 속성 추출
    links = [tr.find_element(By.CSS_SELECTOR, 'a.cmc-link').get_attribute('href') for tr in tr_elements]
    print("Found links:", links)

    for link in links:
        driver.get(link)
        time.sleep(2)

        try:
            # 카테고리 이름 추출
            category_name = link.split('/')[-2]
            results[category_name] = []

            # 특정 클래스 구조를 가진 요소 찾기
            parent_elements = driver.find_elements(By.CSS_SELECTOR, '.sc-4c05d6ef-0.sc-1c5f2868-2.dlQYLv.bszTYj.hide-ranking-number')

            # 각 부모 요소 내의 <p> 태그에서 텍스트 추출
            for parent in parent_elements:
                p_elements = parent.find_elements(By.CSS_SELECTOR, 'p.sc-71024e3e-0')
                for p in p_elements:
                    results[category_name].append(p.text)

            print(f"Found texts for {category_name}: {results[category_name]}")
            driver.back()
            
        except Exception as e:
            print(f"Error processing link {link}: {e}")
            driver.back()

finally:
    driver.quit()

# 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in results.items()]))

# 엑셀 파일로 저장
output_path = os.path.join("/Users/flareon/prometheus/투자추천/크롤링/whitepaper", "category_data.xlsx")
df.to_excel(output_path, index=False)
print(f"Data saved to: {output_path}")